In [ ]:
# Imports for model building and classification
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load the breast cancer dataset
breast_cancer_data = load_breast_cancer()

In [ ]:
# Preview the data
print(breast_cancer_data.keys())
